<a href="https://colab.research.google.com/github/jmellafe/question_deduplication/blob/master/LDA_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import data from the drive 

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_path = '/content/gdrive/My Drive/KTH courses/P4/NLP Language processing/project/data/train.csv' #change dir to your project folder


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Notebook Based on https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24 

In [36]:
import pandas as pd

#Loading gensim and nltk libraries

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from gensim import corpora, models

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
df = pd.read_csv('/content/gdrive/My Drive/KTH courses/P4/NLP Language processing/project/data/train.csv')
print(len(df))
#print(df[:5])
df.head()


404290


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
stemmer = SnowballStemmer("english") # Choose a language
qwords = ['what', 'who', 'when', 'where', 'why', 'which', 'how', 'whom', 'whose']
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    if text:
        text = str(text)
    for token in gensim.utils.simple_preprocess(text):
        if (token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3) or (token in qwords):
            result.append(lemmatize_stemming(token))
    return result

In [29]:
doc_sample = df[df['id'] == 4310]['question1'].values[0]
print(doc_sample)

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))



What are questions which have no answers?
original document: 
['What', 'are', 'questions', 'which', 'have', 'no', 'answers?']

 tokenized and lemmatized document: 
['what', 'question', 'which', 'answer']


In [30]:
processed_q1 = df['question1'].map(preprocess)
processed_q1[:10] 

0    [what, step, step, guid, invest, share, market...
1               [what, stori, kohinoor, noor, diamond]
2             [how, increas, speed, internet, connect]
3                       [whi, mental, lone, how, solv]
4    [which, dissolv, water, quik, sugar, salt, met...
5              [astrolog, capricorn, moon, rise, what]
6                                              [tiago]
7                               [how, good, geologist]
8                                      [when, instead]
9        [motorola, compani, hack, charter, motorolla]
Name: question1, dtype: object

In [32]:
dictionary = gensim.corpora.Dictionary(processed_q1)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 guid
1 india
2 invest
3 market
4 share
5 step
6 what
7 diamond
8 kohinoor
9 noor
10 stori


In [0]:
# Filter out tokens that appear in

  #  less than 15 documents (absolute number) or
  #  more than 0.5 documents (fraction of total corpus size, not absolute number).
  #  after the above two steps, keep only the first 100000 most frequent tokens.

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [35]:
# Gensim doc2bow

# For each document we create a dictionary reporting how many
# words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_q1]
bow_corpus[4310]

bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 6 ("what") appears 1 time.
Word 25 ("which") appears 1 time.
Word 70 ("answer") appears 1 time.
Word 73 ("question") appears 1 time.


TF-IDF

In [47]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
print(len(corpus_tfidf))

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

404290
[(0, 0.42189276289582106),
 (1, 0.18735766840859827),
 (2, 0.3000221398504696),
 (3, 0.28141621302370795),
 (4, 0.335619647105035),
 (5, 0.7087762824565966),
 (6, 0.051873550636748034)]


LDA 

In [59]:
from gensim.models.callbacks import PerplexityMetric
# https://radimrehurek.com/gensim/models/callbacks.html#gensim.models.callbacks.CoherenceMetric


lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=1000, id2word=dictionary, passes=2, workers=6)
# documentation : https://radimrehurek.com/gensim/models/ldamodel.html


/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [60]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.279*"packag" + 0.231*"tast" + 0.145*"what" + 0.118*"door" + 0.050*"how" + 0.050*"tripl" + 0.033*"whi" + 0.015*"talaq" + 0.015*"best" + 0.015*"differ"
Topic: 1 
Words: 0.422*"allow" + 0.318*"resourc" + 0.129*"particular" + 0.076*"what" + 0.049*"configur" + 0.001*"who" + 0.001*"check" + 0.001*"websit" + 0.000*"laptop" + 0.000*"specif"
Topic: 2 
Words: 0.284*"debat" + 0.269*"mexico" + 0.089*"johnson" + 0.080*"gari" + 0.041*"how" + 0.040*"whi" + 0.035*"aeronaut" + 0.030*"stein" + 0.028*"jill" + 0.023*"what"
Topic: 3 
Words: 0.521*"check" + 0.453*"access" + 0.018*"what" + 0.002*"phone" + 0.000*"requir" + 0.000*"account" + 0.000*"websit" + 0.000*"cell" + 0.000*"messag" + 0.000*"allow"
Topic: 4 
Words: 0.191*"ring" + 0.184*"circl" + 0.140*"basi" + 0.093*"how" + 0.073*"greater" + 0.061*"what" + 0.055*"spread" + 0.051*"inner" + 0.042*"whi" + 0.037*"horni"
Topic: 5 
Words: 0.675*"ccna" + 0.184*"which" + 0.031*"exam" + 0.002*"pursu" + 0.001*"rout" + 0.000*"stiff" + 0.000*"witch

In [67]:
# To test a new document
processed_q1[4310]
print(processed_q1[4310])

for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

['what', 'question', 'which', 'answer']

Score: 0.30890506505966187	 
Topic: 0.552*"answer" + 0.111*"what" + 0.070*"phase" + 0.065*"whi" + 0.064*"upvot" + 0.054*"quora" + 0.051*"annoy" + 0.013*"menu" + 0.011*"like" + 0.005*"differ"

Score: 0.27548664808273315	 
Topic: 0.474*"question" + 0.227*"ask" + 0.201*"what" + 0.051*"best" + 0.016*"how" + 0.014*"abstract" + 0.005*"whi" + 0.005*"interview" + 0.003*"who" + 0.002*"differ"

Score: 0.2004806250333786	 
Topic: 0.984*"which" + 0.007*"better" + 0.002*"phone" + 0.001*"peopl" + 0.001*"mobil" + 0.001*"what" + 0.000*"appl" + 0.000*"think" + 0.000*"samsung" + 0.000*"suicid"

Score: 0.01592312380671501	 
Topic: 0.320*"question" + 0.174*"whi" + 0.149*"quora" + 0.118*"peopl" + 0.104*"answer" + 0.098*"easili" + 0.020*"when" + 0.008*"how" + 0.005*"chromosom" + 0.001*"who"


Different references
*  https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50
* http://papers.www2017.com.au.s3-website-ap-southeast-2.amazonaws.com/proceedings/p1221.pdf
* http://www.erogol.com/duplicate-question-detection-deep-learning/
* https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
* http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf
* http://www.erogol.com/duplicate-question-detection-deep-learning/
